In [80]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
%gui qt

import copy
from scipy.io import loadmat

In [81]:
#load relevant header data
rawHeaderData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\rawframeHeader.mat'))['rawframeHeader'][0]

In [82]:
#preprocessing
headerStream = np.array([])
for number in rawHeaderData[0]:
    headerStream = np.uint8(np.append(headerStream,number[0]))

In [83]:
def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

In [84]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0



In [85]:
header = readHeader(headerStream)

In [86]:
header

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8),
 'packetLength': array([1146], dtype=uint32),
 'frameNumber': array([11758], dtype=uint32),
 'numTLVs': array([3], dtype=uint16)}

In [87]:
dataLength = int(header['packetLength'] - frameHeaderLength)

In [88]:
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\tlvData.mat'))['tlvData'][0][0]

In [89]:
tlvStream = np.frombuffer(tlvData, dtype = 'uint8')

In [90]:
#tlv header
index = 0
#tlv header parsing
tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)

In [91]:
print(tlvType)
print(tlvLength)

[6]
[888]


In [92]:
index += tlvHeaderLengthInBytes
tlvDataLength = tlvLength - tlvHeaderLengthInBytes

In [105]:
if tlvType == 6: 
     numberOfPoints = tlvDataLength/pointLengthInBytes
     p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
     pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

In [106]:
if not(pointCloud is None):
    posX = np.multiply(pointCloud[0,:], np.sin(pointCloud[1,:]))
    posY = np.multiply(pointCloud[0,:], np.cos(pointCloud[1,:]))


In [107]:
posX

array([-1.8993163 ,  3.1347146 ,  3.0488634 ,  0.76953757,  0.78524244,
        0.9737006 ,  0.65240383,  0.6665865 ,  0.68076926,  0.6949519 ,
        0.7091346 ,  0.54375   ,  0.5563953 ,  0.581686  ,  0.5943314 ,
        0.60697675,  0.6322674 ,  1.1760174 ,  0.4326841 ,  0.44377854,
        0.454873  ,  0.46596748,  0.47706196,  0.48815638,  0.49925086,
        0.51034534,  0.5214398 ,  0.5325343 ,  1.0317851 ,  0.37173626,
        0.38126794,  0.39079964,  0.40033135,  0.40986305,  0.41939473,
        0.42892644,  0.43845814,  0.44798985,  0.31039032,  0.31834906,
        0.3263078 ,  0.3342665 ,  0.34222525,  0.35018396,  0.3581427 ,
        0.3661014 ,  0.37406015,  0.3820189 ,  0.26146653,  0.26784378,
        0.27422103,  0.3061072 ,  0.20592311,  0.22986765,  0.153382  ],
      dtype=float32)

In [104]:
posY

array([1.4573966, 2.5725937, 2.6737814, 2.2669847, 2.3132496, 2.8684294,
       2.1506875, 2.1974416, 2.2441957, 2.2909496, 2.3377037, 2.0293026,
       2.0764956, 2.1708817, 2.218075 , 2.265268 , 2.3596542, 4.3889565,
       1.8556808, 1.9032623, 1.9508439, 1.9984255, 2.0460072, 2.0935886,
       2.14117  , 2.1887517, 2.2363334, 2.283915 , 4.4250846, 1.8688443,
       1.9167633, 1.9646825, 2.0126016, 2.0605206, 2.1084397, 2.1563587,
       2.2042778, 2.252197 , 1.8800066, 1.9282118, 1.9764171, 2.0246224,
       2.0728278, 2.121033 , 2.1692383, 2.2174437, 2.265649 , 2.3138542,
       1.9860355, 2.0344753, 2.0829153, 2.325115 , 2.0907724, 2.3338854,
       2.340157 ], dtype=float32)